Задача: породить такой нетерминал, который бы мог матчить слова по их эмбеддингу.

Проблема: придётся переписывать nltk'шные парсеры

Решение: следуя примеру Джонатана, на лету будем дописывать в грамматику новые правила

https://groups.google.com/forum/#!topic/nltk-users/4nC6J7DJcOc/discussion

# Стандартная грамматика

In [2]:
gram_text = """
S -> TURN COLOR_OF_DEVICE COLOR
COLOR -> 'синий' | 'зеленый'
TURN -> 'включи' | 'сделай' | 'поставь'
COLOR_OF_DEVICE -> COLOR_N | COLOR_N DEVICE
COLOR_N -> 'цвет' | 'свет'
DEVICE -> 'лампочки'
"""

In [3]:
from nltk import CFG
from nltk.parse import BottomUpLeftCornerChartParser

In [4]:
grammar = CFG.fromstring(gram_text)

In [5]:
grammar

<Grammar with 11 productions>

In [6]:
parser = BottomUpLeftCornerChartParser(grammar)

In [7]:
def try_parse(text, parser):
    try:
        tree = parser.parse_one(text.lower().split())
    except ValueError:
        return None
    return tree

In [53]:
p = try_parse('сделай цвет зеленый', parser)

In [13]:
print(p)

(S (TURN сделай) (COLOR_OF_DEVICE (COLOR_N цвет)) (COLOR зеленый))


# Эмбеддинги

In [22]:
import compress_fasttext
small_model = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
    'https://github.com/avidale/compress-fasttext/releases/download/v0.0.1/ft_freqprune_100K_20K_pq_100.bin'
)
#print(small_model['спасибо'])

In [23]:
small_model.similarity('синий', 'голубой')

0.664403554367599

In [60]:
small_model.similarity('зеленый', 'цвет')

0.576825335473742

In [46]:
small_model.init_sims()

# Обогащаем

In [61]:
min_sims = {
    'COLOR': 0.6,
}

In [38]:
from collections import defaultdict

In [70]:
keywords = defaultdict(list)
nonterms= {}

for p in grammar.productions():
    lhs = p.lhs()
    if lhs.symbol() not in min_sims:
        continue
    if not isinstance(lhs, nltk.grammar.Nonterminal):
        continue
    nonterms[lhs.symbol()] = lhs
    rhs = p.rhs()
    if not isinstance(rhs, tuple):
        continue
    if len(rhs) != 1: # todo: work with multiword expressions as well !
        continue
    keywords[lhs.symbol()].append(rhs[0])

In [73]:
type(nonterms['COLOR'])

nltk.grammar.Nonterminal

In [47]:
key_vectors = {
    k: [small_model.word_vec(w, use_norm=True) for w in v]
    for k, v in keywords.items()
}

In [34]:
import nltk.grammar

In [54]:
import numpy as np

In [84]:
vec.dot(new_vec)

0.6302224

In [67]:
text2 = 'сделай цвет лампы желтый'

toks = text2.split()

new_productions = []

for tok in toks:
    new_vec = small_model.word_vec(tok, use_norm=True)
    for key, vecs in key_vectors.items():
        for vec in vecs:
            if np.dot(new_vec, vec) > min_sims[key]:
                print(key, tok)
                new_productions.append([key, tok])
                break

COLOR желтый


In [69]:
grammar.unicode_repr()

'<Grammar with 11 productions>'

In [83]:
%%time

new_grammar = CFG(
    productions=list(grammar.productions()) + [
        nltk.grammar.Production(nonterms[l], (r,)) for l, r in new_productions
    ],
    start=grammar.start(),
)
new_grammar

Wall time: 500 µs


<Grammar with 12 productions>

In [82]:
%%time
new_parser = BottomUpLeftCornerChartParser(new_grammar)

Wall time: 0 ns


In [81]:
p = try_parse('сделай цвет желтый', new_parser)
print(p)

(S (TURN сделай) (COLOR_OF_DEVICE (COLOR_N цвет)) (COLOR желтый))


In [90]:
from importlib import reload

In [105]:
import word_mover_grammar
reload(word_mover_grammar);

In [108]:
hacked_grammar = word_mover_grammar.grammar_tools.HackedGrammar(
    grammar=grammar, 
    w2v=small_model, 
    min_sims={'COLOR': 0.5},
)

In [111]:
text = 'сделай цвет лампочки фиолетовый'
new_parser = BottomUpLeftCornerChartParser(hacked_grammar.enrich_grammar(text))

print(try_parse(text, new_parser))

COLOR цвет
COLOR фиолетовый
(S
  (TURN сделай)
  (COLOR_OF_DEVICE (COLOR_N цвет) (DEVICE лампочки))
  (COLOR фиолетовый))
